### Here is Emilio's Attempt to follow the tutorial
#### https://machinelearningmastery.com/how-to-train-an-object-detection-model-with-keras/
#### but with Massimo's data

# Import your dependencies 
import xml.etree.ElementTree as ElementTree
# load and parse the file
filename = "/home/jovyan/sample_data/annots/201503.20150602.115901553.62766.orig.xml"
tree = ElementTree.parse(filename)
# get the root of the document
root = tree.getroot()
# show the root 
type(root)

# With findall extract the values that define each bounding box
for box in root.findall('.//bndbox'):
    xmin = int(box.find('xmin').text)
    ymin = int(box.find('ymin').text)
    xmax = int(box.find('xmax').text)
    ymax = int(box.find('ymax').text)
    coors = [xmin, ymin, xmax, ymax]
# print the values
print(coors)

# extract image dimensions
width = int(root.find('.//size/width').text)
height = int(root.find('.//size/height').text)
print(width)
print(height)

#### Exercise above, function below

In [1]:
# function to extract bounding boxes from an annotation file

from xml.etree import ElementTree

def extract_boxes(filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = list()
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        coors = [xmin, ymin, xmax, ymax]
        boxes.append(coors)
    # extract image dimensions
    width = int(root.find('.//size/width').text)
    height = int(root.find('.//size/height').text)
    return boxes, width, height

In [2]:
# load and parse the file
filename = "/home/jovyan/sample_data/annots/201503.20150602.115901553.62766.orig.xml"
# extract details form annotation file
boxes, w, h = extract_boxes(filename)
# summarize extracted details
print(boxes, w, h)

[[93, 39, 110, 58]] 1360 1024


#### Develop sand dollar training object

In [3]:
## This means that a new class must be defined that extends the mrcnn.utils.Dataset class 
## and defines a function to load the dataset, with any name you like such as load_dataset(), 
## and override two functions, one for loading a mask called load_mask() 
## and one for loading an image reference (path or URL) called image_reference().

# class that defines and loads the kangaroo dataset
class KangarooDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # ...

    # load the masks for an image
    def load_mask(self, image_id):
        # ...

    # load an image reference
    def image_reference(self, image_id):
        # ...

IndentationError: expected an indented block (<ipython-input-3-4af6eaed7226>, line 13)

In [6]:
# prepare the dataset
train_set = sand_dollar_dataset()
train_set.load_dataset(...)
train_set.prepare()

NameError: name 'sand_dollar_dataset' is not defined

##### Classes are defined by calling the built-in add_class() function and specifying the ‘source‘ (the name of the dataset), the ‘class_id‘ or integer for the class (e.g. 1 for the first lass as 0 is reserved for the background class), and the ‘class_name‘ (e.g. ‘kangaroo‘).

In [5]:
# define one class
self.add_class("dataset", 1, "sand_dollar")


NameError: name 'self' is not defined

##### Objects are defined by a call to the built-in add_image() function and specifying the ‘source‘ (the name of the dataset), a unique ‘image_id‘ (e.g. the filename without the file extension like ‘00001‘), and the path for where the image can be loaded (e.g. ‘kangaroo/images/00001.jpg‘).

In [ ]:
# add to dataset
self.add_image('dataset', image_id='00001', 
               path='/home/jovyan/sample_data/images/201503.20150602.115901553.62766.orig.jpg', 
               annotation='/home/jovyan/sample_data/annots/201503.20150602.115901553.62766.orig.xml')

In [7]:
# load the dataset definitions
def load_dataset(self, dataset_dir):
    # define one class
    self.add_class("dataset", 1, "sand_dollar")
    # define data locations
    dataset_dir = '/home/jovyan/sample_data'
    images_dir = dataset_dir + '/images/'
    annotations_dir = dataset_dir + '/annots/'
    # find all images
    for filename in listdir(images_dir):
        # extract image id
        image_id = filename[:-4]
        # skip bad images
        if image_id in ['00090']:
            continue
        img_path = images_dir + filename
        ann_path = annotations_dir + image_id + '.xml'
        # add to dataset
        self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

##### First, we must load the annotation file for the image_id. This involves first retrieving the ‘image info‘ dict for the image_id, then retrieving the annotations path that we stored for the image via our prior call to add_image(). We can then use the path in our call to extract_boxes() developed in the previous section to get the list of bounding boxes and the dimensions of the image.

In [9]:
# get details of image
info = self.image_info[image_id]
# define box file location
path = info['annotation']
# load XML
boxes, w, h = self.extract_boxes(path)

NameError: name 'self' is not defined

##### A mask is a two-dimensional array with the same dimensions as the photograph with all zero values where the object isn’t and all one values where the object is in the photograph. 
##### We can achieve this by creating a NumPy array with all zero values for the known size of the image and one channel for each bounding box.

In [ ]:
# create one array for all masks, each on a different channel
masks = zeros([h, w, len(boxes)], dtype='uint8')

In [ ]:
## These can be used directly to define row and column ranges in the array that can then be marked as 1.
# create masks
for i in range(len(boxes)):
    box = boxes[i]
    row_s, row_e = box[1], box[3]
    col_s, col_e = box[0], box[2]
    masks[row_s:row_e, col_s:col_e, i] = 1

In [ ]:
#All objects have the same class in this dataset.
#We can retrieve the class index via the ‘class_names‘ dictionary, then add it to a list to be returned alongside the masks.
self.class_names.index('sand_dollar')

In [8]:
## LOAD the complete load_mask() function is listed below.
# load the masks for an image
def load_mask(self, image_id):
    # get details of image
    info = self.image_info[image_id]
    # define box file location
    path = info['annotation']
    # load XML
    boxes, w, h = self.extract_boxes(path)
    # create one array for all masks, each on a different channel
    masks = zeros([h, w, len(boxes)], dtype='uint8')
    # create masks
    class_ids = list()
    for i in range(len(boxes)):
        box = boxes[i]
        row_s, row_e = box[1], box[3]
        col_s, col_e = box[0], box[2]
        masks[row_s:row_e, col_s:col_e, i] = 1
        class_ids.append(self.class_names.index('kangaroo'))
    return masks, asarray(class_ids, dtype='int32')

##### A mask is a two-dimensional array with the same dimensions as the photograph with all zero values where the object isn’t and all one values where the object is in the photograph. 
##### We can achieve this by creating a NumPy array with all zero values for the known size of the image and one channel for each bounding box.

In [ ]:

#This function is responsible for returning the path
#or URL for a given ‘image_id‘, which we know is just the ‘path‘ property on the ‘image info‘ dict
# load an image reference
def image_reference(self, image_id):
    info = self.image_info[image_id]
    return info['path']

# FIX THIS

In [ ]:
#
#split into train and test set
from os import listdir
from xml.etree import ElementTree
from numpy import zeros
from numpy import asarray
from mrcnn.utils import Dataset
 
# class that defines and loads the kangaroo dataset
class KangarooDataset(Dataset):
    # load the dataset definitions
    def load_dataset(self, dataset_dir, is_train=True):
        # define one class
        self.add_class("dataset", 1, "sand_dollar")
        # define data locations
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        # find all images
        for filename in listdir(images_dir):
            # extract image id
            image_id = filename[:-4]
            # skip bad images
            if image_id in ['00090']:
                continue
            # skip all images after 150 if we are building the train set
            if is_train and int(image_id) >= 150:
                continue
            # skip all images before 150 if we are building the test/val set
            if not is_train and int(image_id) < 150:
                continue
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            # add to dataset
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)
 
    # extract bounding boxes from an annotation file
    def extract_boxes(self, filename):
        # load and parse the file
        tree = ElementTree.parse(filename)
        # get the root of the document
        root = tree.getroot()
        # extract each bounding box
        boxes = list()
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        # extract image dimensions
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        return boxes, width, height
 
    # load the masks for an image
    def load_mask(self, image_id):
        # get details of image
        info = self.image_info[image_id]
        # define box file location
        path = info['annotation']
        # load XML
        boxes, w, h = self.extract_boxes(path)
        # create one array for all masks, each on a different channel
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        # create masks
        class_ids = list()
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('sand_dollar'))
        return masks, asarray(class_ids, dtype='int32')
 
    # load an image reference
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']
 
# train set
train_set = KangarooDataset()
train_set.load_dataset('sand_dollar', is_train=True)
train_set.prepare()
print('Train: %d' % len(train_set.image_ids))
 
# test/val set
test_set = KangarooDataset()
test_set.load_dataset('kangaroo', is_train=False)
test_set.prepare()
print('Test: %d' % len(test_set.image_ids))